In [1]:
import pandas as pd
import numpy as np
import itertools
import random
import ast
import matplotlib
import matplotlib.pyplot as plt
from scipy.stats import spearmanr, wilcoxon, friedmanchisquare, kendalltau, pearsonr
#from matplotlib import animation
#from mpl_toolkits import mplot3d
from scipy.spatial import distance
from pyxdameraulevenshtein import damerau_levenshtein_distance
from matplotlib.ticker import PercentFormatter

pd.set_option('display.max_rows', 500)

### Prepare Data

In [2]:
# read data

df = pd.read_csv('automatica_task_environments_all.csv', header=0,
                 skip_blank_lines=True, skiprows=[])

In [3]:
df

,ID,sequence_original,sequence_no_duplicates,sequence_clustered_silverware,sequence_clustered_drawers,coordinates_original,coordinates_clustered_silverware,coordinates_clustered_drawers,start_coordinates,strong_k,...,food_k,containment,error_original,length_original,error_no_duplicates,length_no_duplicates,error_clustered_silverware,length_clustered_silverware,error_clustered_drawers,length_clustered_drawers
0,a1,pocgkr,pocgkr,pocgz,pocgba,"p: (0.008034,0.957082,0.6890539999999999);o: (...","p: (0.008034,0.957082,0.6890539999999999);o: (...","p: (0.008034,0.957082,0.6890539999999999);o: (...","[-0.451354,-0.413918,0.156247],[0.513,-0.531,0...",t,...,0,0,0.723,6,0.723,6,0.666,5,0.723,5
1,a3,cgwpcffkkss,cgwpcfks,cgwpdz,cgwpdb,"c: (-0.525007,0.923283,0.438);g: (-0.595001,1....","c: (-0.525007,0.923283,0.438);g: (-0.595001,1....","c: (-0.525007,0.923283,0.438);g: (-0.595001,1....","[-0.451354,-0.413918,0.156247],[0.513,-0.531,0...",t,...,0,0,0.838,11,0.785,8,0.723,6,0.723,6
2,a5,kfsfksppwwggg,kfsfkspwg,zpwg,bpwg,"k: (-0.07699199999999999,0.733425,0.531662);f:...","k: (-0.07699199999999999,0.733425,0.531662);f:...","k: (-0.07699199999999999,0.733425,0.531662);f:...","[-0.451354,-0.413918,0.156247],[0.513,-0.531,0...",t,...,0,0,0.862,13,0.806,9,0.605,4,0.605,4
3,a11,ppfkswwkfsococggg,pfkswkfsococg,pzwzocndg,pbwbocndg,"p: (0.008034,0.957082,0.6890539999999999);p: (...","p: (0.008034,0.957082,0.6890539999999999);p1: ...","p: (0.008034,0.957082,0.6890539999999999);p1: ...","[-0.451354,-0.413918,0.156247],[0.513,-0.531,0...",t,...,0,0,0.892,17,0.862,13,0.806,9,0.806,9
4,a13,wptgkfsoc,wptgkfsoc,wptgzoc,wptgboc,"w: (-0.196022,1.35572,0.466005);p: (0.008034,0...","w: (-0.196022,1.35572,0.466005);p: (0.008034,0...","w: (-0.196022,1.35572,0.466005);p: (0.008034,0...","[-0.451354,-0.413918,0.156247],[0.513,-0.531,0...",t,...,0,0,0.806,9,0.806,9,0.756,7,0.756,7
5,a16,pppwwwfffkkkggg,pwfkg,pwzg,pwbg,"p: (0.008034,0.957082,0.6890539999999999);p: (...","p: (0.008034,0.957082,0.6890539999999999);p1: ...","p: (0.008034,0.957082,0.6890539999999999);p1: ...","[-0.451354,-0.413918,0.156247],[0.513,-0.531,0...",t,...,0,0,0.878,15,0.666,5,0.605,4,0.605,4
6,a19,ppoowwcceerr,powcer,powcz,powca,"p: (0.008034,0.957082,0.6890539999999999);p: (...","p: (0.008034,0.957082,0.6890539999999999);p1: ...","p: (0.008034,0.957082,0.6890539999999999);p1: ...","[-0.451354,-0.413918,0.156247],[0.513,-0.531,0...",t,...,0,0,0.851,12,0.723,6,0.666,5,0.666,5
7,a22,cooppreerkk,coprerk,copz,copab,"c: (-0.7949970000000001,0.9232809999999999,0.4...","c: (-0.7949970000000001,0.9232809999999999,0.4...","c: (-0.7949970000000001,0.9232809999999999,0.4...","[-0.451354,-0.413918,0.156247],[0.513,-0.531,0...",t,...,0,0,0.838,11,0.756,7,0.605,4,0.666,4
8,a26,wwoppoffkkssskf,wopofkkskf,wopnz,wopnb,"w: (-0.196022,1.35572,0.466005);w: (-0.0570049...","w: (-0.196022,1.35572,0.466005);w1: (-0.057004...","w: (-0.196022,1.35572,0.466005);w1: (-0.057004...","[-0.451354,-0.413918,0.156247],[0.513,-0.531,0...",t,...,0,0,0.878,15,0.824,10,0.666,5,0.666,5
9,a29,opwpwgcoc,opwpwgcoc,opwqxgcnd,opwqxgcnd,"o: (-0.185,0.928,0.48103500000000005);p: (0.00...","o: (-0.185,0.928,0.48103500000000005);p: (0.00...","o: (-0.185,0.928,0.48103500000000005);p: (0.00...","[-0.451354,-0.413918,0.156247],[0.513,-0.531,0...",t,...,0,0,0.806,9,0.806,9,0.806,9,0.806,9


### Define functions for prediction

In [4]:
def predict_sequence(objects, coordinates, start_coordinates, c, k, dimension=[3,]):
    ''' Predicts sequence based on required objects, object coordinates, start coordinates of subject,
        parameters (c+k) and dimensionality.
        Input: Objects, object coordinates, start coordinates, c, k, dimension
        Output: Sequence of objects as str
    '''
    prediction = []
    possible_items = dict.fromkeys(objects, 0) # generate dict from object list
    coord_index = 0
    start_coords = start_coordinates
    coords = coordinates
    new_coords = {}
    new_start_coords = []
    
    if dimension[0] == 3: # no changes if 3D
        new_coords = coords
        new_start_coords = start_coords
        
    elif dimension[0] == 2: # 2D: remove obsolete coordinate
        if dimension[1] == 'xy':
            new_coords = {key: value[:-1] for key, value in coords.items()}
            new_start_coords = [x[:-1] for x in start_coords]
                
        elif dimension[1] == 'xz':
            new_start_coords = [[x[0], x[-1]] for x in start_coords]
            
            for key, value in coords.items():
                new_value = (value[0], value[-1])
                new_coords[key] = new_value
        
        elif dimension[1] == 'yz':
            new_coords = {key: value[1:] for key, value in coords.items()}
            new_start_coords = [x[1:] for x in start_coords]
                
    elif dimension[0] == 1: # 1D: choose appropriate coordinate
        if dimension[1] == 'x':
            new_coords = {key: value[0] for key, value in coords.items()}
            new_start_coords = [x[0] for x in start_coords]
        
        elif dimension[1] == 'y':
            new_coords = {key: value[1] for key, value in coords.items()}
            new_start_coords = [x[1] for x in start_coords]
        
        elif dimension[1] == 'z':
            new_coords = {key: value[2] for key, value in coords.items()}
            new_start_coords = [x[2] for x in start_coords]
    
    while bool(possible_items) == True: # while dict not empty
        for obj in possible_items.keys():
            possible_items[obj] = ((distance.euclidean(
                                new_start_coords[coord_index], 
                                new_coords[obj])
                                ) ** k[obj]) * c[obj]
        #print(possible_items)                      
        minval = min(possible_items.values())
        minval = [k for k, v in possible_items.items() if v == minval]
        minval = random.choice(minval) # choose prediction randomly if multiple items have same cost
        prediction.append(minval)
        del possible_items[minval]
        coord_index += 1
        
    return prediction

In [5]:
def get_median(objects, coordinates, start_coordinates, c, k, dimension, sequence,n=10):
    ''' Returns average edit distance (Damerau-Levenshtein) for 100 trials of sequence prediction.
    '''
    edit_list = []

    for x in range(0,n):
        #print(objects)
        result = ''.join(predict_sequence(objects, coordinates, start_coordinates, c, k, dimension))
        dl = damerau_levenshtein_distance(sequence,result)
        edit_list.append(dl)
    
    #avg = np.mean(edit_list)
    median = np.median(edit_list)
    return median

In [6]:
def get_avg_editdist(data, dimensions=[[1,'x'],[1,'y'],[1,'z'],[2,'xy'],[2,'xz'],[2,'yz'],[3,'xyz']],n=10,
                    seq='sequence_original', coords='coordinates_original', error='error_original'):
    ''' Calculates average edit distance for all combinations of parameters (c, k, dimension).
        Input: Dataframe with objects, coordinates, start coordinates, object categories
        Output: Dataframe with edit distance results (col name: parameters used)
    '''
    results = pd.DataFrame()
        
    for row in range(0,len(data)):
        objects = list(data.at[row,seq])
        strong_k = list(data.at[row,'strong_k'].split(','))
        mid_k = list(data.at[row,'mid_k'].split(','))
        #food_k = list(data.at[row,'food_k'].split(','))
        food_k = []
        coordinates = {key: ast.literal_eval(value) for key, value in (elem.split(': ') for elem in data.at[row,coords].split(';'))}
        #print(coordinates)
        start_coordinates = list(ast.literal_eval(data.at[row,'start_coordinates']))
        sequence = str(data.at[row,seq])
        c1 = {obj: 1.2 for obj in objects}
        k1 = {obj: 1.0 for obj in objects}
        
        #for k2 in np.arange(1.1,2.0,0.1):
            #k_food = round(k2,2)
            #k1 = {obj: k_food if obj in food_k else 1.0 for obj in objects}
        
        for k in np.arange(0,0.9,0.1):
            k_strong = round(k,2)
            k_mid = round(k + 0.1,2)
            k1 = {obj: k_strong if obj in strong_k else k_mid if obj in mid_k else round(k1[obj],2) for obj in objects}
            
                #for c in np.arange(1.0,2.0,0.1):
                #    c = round(c, 1)
                #    c1 = {obj: 1.2 for obj in objects}
                
            for dim in dimensions:                
                # get average edit distance
                median = get_median(objects, coordinates, start_coordinates, c1, k1, dim, sequence, n)
                median = median / len(sequence)
                #params = 'c: ' + str(c) + '; k: ' + str(k_strong) + ',' + str(k_mid) + ',' + str(k_food) + '; ' + str(dim[1])
                params = 'c: 1.2' + '; k: ' + str(k_strong) + ',' + str(k_mid) + '; ' + str(dim[1])
                results.at[row,params] = median
                        
        results.at[row,'sequence'] = sequence
        results.at[row,'error'] = data.at[row,error]
        results.at[row,'ID'] = data.at[row,'ID']
                    
    return results

### Calculate edit distances, create df for original sequences

In [14]:
# ~3 min for n=100
%time results_original = get_avg_editdist(df,n=100)

CPU times: user 3min 23s, sys: 226 ms, total: 3min 23s
Wall time: 3min 23s


In [15]:
results_original
#results_original.T.to_csv('results/results_original_all_n100_2020-11-03.csv', header=True, index=True)

,"c: 1.2; k: 0.0,0.1; x","c: 1.2; k: 0.0,0.1; y","c: 1.2; k: 0.0,0.1; z","c: 1.2; k: 0.0,0.1; xy","c: 1.2; k: 0.0,0.1; xz","c: 1.2; k: 0.0,0.1; yz","c: 1.2; k: 0.0,0.1; xyz","c: 1.2; k: 0.1,0.2; x","c: 1.2; k: 0.1,0.2; y","c: 1.2; k: 0.1,0.2; z",...,"c: 1.2; k: 0.8,0.9; x","c: 1.2; k: 0.8,0.9; y","c: 1.2; k: 0.8,0.9; z","c: 1.2; k: 0.8,0.9; xy","c: 1.2; k: 0.8,0.9; xz","c: 1.2; k: 0.8,0.9; yz","c: 1.2; k: 0.8,0.9; xyz",sequence,error,ID
0,0.666667,0.833333,0.833333,0.833333,0.833333,0.833333,0.833333,0.666667,0.833333,0.833333,...,0.666667,0.833333,0.833333,0.666667,0.833333,0.833333,0.666667,pocgkr,0.723,a1
1,0.818182,0.818182,0.818182,0.818182,0.818182,0.818182,0.818182,0.818182,0.818182,0.818182,...,0.727273,0.818182,0.636364,0.818182,0.727273,0.818182,0.818182,cgwpcffkkss,0.838,a3
2,0.615385,0.769231,0.692308,0.692308,0.615385,0.769231,0.692308,0.615385,0.769231,0.692308,...,0.615385,0.615385,0.692308,0.538462,0.615385,0.615385,0.538462,kfsfksppwwggg,0.862,a5
3,0.647059,0.647059,0.705882,0.647059,0.705882,0.647059,0.647059,0.647059,0.647059,0.705882,...,0.647059,0.647059,0.647059,0.647059,0.764706,0.647059,0.647059,ppfkswwkfsococggg,0.892,a11
4,0.888889,0.777778,0.888889,0.666667,0.888889,0.777778,0.666667,0.888889,0.666667,0.888889,...,0.777778,0.888889,0.666667,1.000000,0.777778,0.888889,1.000000,wptgkfsoc,0.806,a13
5,0.800000,0.800000,0.733333,0.800000,0.800000,0.800000,0.800000,0.800000,0.800000,0.733333,...,0.800000,0.866667,0.733333,0.800000,0.800000,0.866667,0.800000,pppwwwfffkkkggg,0.878,a16
6,0.833333,0.833333,0.833333,0.750000,0.833333,0.833333,0.750000,0.833333,0.833333,0.833333,...,0.750000,0.833333,0.833333,0.666667,0.750000,0.833333,0.666667,ppoowwcceerr,0.851,a19
7,0.818182,0.636364,0.727273,0.636364,0.818182,0.636364,0.636364,0.818182,0.636364,0.727273,...,0.818182,0.909091,0.636364,0.818182,0.818182,0.818182,0.818182,cooppreerkk,0.838,a22
8,0.866667,0.733333,0.733333,0.733333,0.866667,0.733333,0.733333,0.866667,0.733333,0.733333,...,0.800000,0.866667,0.666667,0.866667,0.733333,0.866667,0.866667,wwoppoffkkssskf,0.878,a26
9,0.666667,0.666667,0.666667,0.555556,0.666667,0.666667,0.555556,0.666667,0.666667,0.666667,...,0.666667,0.666667,0.666667,0.555556,0.666667,0.666667,0.555556,opwpwgcoc,0.806,a29


### Get lowest error, compare edit distances for all variants

In [16]:
def get_lowest_error(results):
    ''' Returns lowest error in dataframe and index of lowest error.
    '''
    for col in list(results):
        if col != 'sequence' and col != 'error' and col != 'ID':
            results.loc['mean',col] = results[col].mean()
            #results.loc['median',col] = results[col].median()
    lowest = min(results.loc['mean'])
    mean = list(results.loc['mean'])
    
    return lowest, results.columns[(results.loc['mean'] == lowest)], mean, results

In [17]:
# original sequences

lowest, lowest_idx, list_mean, results_mean = get_lowest_error(results_original)
lowest, lowest_idx, np.mean(results_original['error'])

(0.7108442291242113,
 Index(['c: 1.2; k: 0.7,0.8; z', 'c: 1.2; k: 0.8,0.9; z'], dtype='object'),
 0.8318749999999998)

In [31]:
distances = [0.33333333333333337, 0.5, 0.6666666666666667, 0.5384615384615384, 0.5555555555555556, 0.19999999999999996, 0.5, 0.5714285714285714, 0.6, 0.4444444444444444, 0.6, 0.625, 0.5384615384615384, 0.5, 0.33333333333333337, 0.5, 0.7272727272727273, 0.5, 0.5714285714285714, 0.6, 0.6363636363636364, 0.625, 0.6, 0.5882352941176471, 0.375, 0.375, 0.5, 0.5555555555555556, 0.5714285714285714, 0.19999999999999996, 0.25, 0.6923076923076923, 0.5555555555555556, 0.7, 0.4285714285714286, 0.5, 0.6363636363636364, 0.5, 0.6363636363636364, 0.33333333333333337, 0.33333333333333337, 0.4, 0.6, 0.4285714285714286, 0.6, 0.25, 0.4, 0.5555555555555556, 0.5, 0.5, 0.16666666666666663, 0.4, 0.6666666666666667, 0.4285714285714286, 0.625, 0.5454545454545454, 0.5555555555555556, 0.5714285714285714, 0.6, 0.4, 0.25, 0.7692307692307692, 0.7777777777777778, 0.4285714285714286, 0.5, 0.19999999999999996, 0.46153846153846156, 0.5384615384615384, 0.5, 0.5714285714285714, 0.5, 0.25, 0.6666666666666667, 0.7, 0.4444444444444444, 0.6923076923076923, 0.6363636363636364, 0.4285714285714286, 0.6363636363636364, 0.375]

### Plot best model

In [34]:
#%matplotlib inline
%matplotlib qt

IDs = df['ID']

error_original = df['error_original']

seqs_original = results_mean['sequence'][:-1].values

res_original = results_original['c: 1.2; k: 0.7,0.8; z'][:-1].values

x_original = [x for x in range (0,len(seqs_original))]

# plot scatter + lines for all simulations
plt.scatter(x_original, res_original, marker='o', s=20, c='blue', alpha=0.6, 
            label=str('model-generated avg.: ') + str(round(lowest,2)))
plt.plot(x_original, res_original, c='blue', alpha=0.6)

# plot CPT baseline
plt.scatter(x_original, distances, marker='o', s=20, c='darkgreen', alpha=0.3, 
            label=str('CPT baseline avg.: ') + str(round(np.mean(distances),2)))
plt.plot(x_original, distances, '-', c='darkgreen', alpha=0.3)
#plt.plot((x_original,x_original),(res_original,distances), '--', c='darkgreen', alpha=0.6)
plt.fill_between(x_original, distances, alpha=0.2, color='darkgreen')

# plot random baseline
plt.scatter(x_original, error_original, c='red', s=20, marker='o', alpha=0.3, 
            label=str('guessing baseline avg.: ') + str(round(np.mean(results_original['error']),2)))
plt.plot(x_original, error_original, c='red', alpha=0.3)#, 
         #label=str('guessing baseline avg.: ' + str(round(np.mean(results_original['error']),2))))
plt.fill_between(x_original, error_original, [1.0 * len(error_original)], color='red', alpha=0.2)


plt.xticks(x_original, labels=IDs, rotation=90, fontsize=7)
#plt.xticklabels(IDs, rotation=90, fontsize=6)

plt.ylabel('edit distance', fontsize=10)
plt.xlabel('sequence', fontsize=10)
plt.ylim(0.0, 1.0)
plt.title('best model (automatica), c:1.2; k: 0.7,0.8; z')

plt.legend(fontsize=8, framealpha=0.9, loc='lower left')

#plt.savefig('plot_median_editdist_individualerrors_diff.png', bbox_inches='tight')
plt.show()

### Calculate, compare, plot percentual change from baseline to prediction

In [23]:
def calculate_percentual_difference(var1, var2):
    
    return [(((np.abs(x - y)) / ((x + y) / 2)) * 100) for x,y in zip(var1, var2)]

In [24]:
def calculate_percentage_change(var1, var2):
    
    return [(((x - y) / np.abs(x)) * 100) if x != 0 else (((0.01 - y) / 0.01) * 100) for x,y in zip(var1, var2)]
    #return [(((x - y) / np.abs(x)) * 100) for x,y in zip(var1, var2) if x != 0]

In [67]:
percent_original = calculate_percentual_difference(res_original, error_original)

In [ ]:
original_change = calculate_percentage_change(res_original, error_original)

In [26]:
np.nanmedian(original_change)

(-17.923809523809528,
 -17.714285714285733,
 -10.999999999999991,
 -8.450000000000003)

In [61]:
%matplotlib qt

plt.rc('ytick', labelsize=7)

fig2, axs2 = plt.subplots(nrows=4, ncols=1, sharex=True, sharey=True)

axs2[0].scatter(x_original, original_change, label='median: -17.92', c='blue', alpha=0.6)
axs2[0].plot(x_original, original_change, c='black', alpha=0.5, linestyle='--')
axs2[0].axhline(y=0, xmin=0, xmax=1, c='red')

axs2[1].scatter(x_noduplicates, noduplicates_change, label='median: -17.71', c='darkgoldenrod', alpha=0.6)
axs2[1].plot(x_noduplicates, noduplicates_change, c='black', alpha=0.5, linestyle='--')
axs2[1].axhline(y=0, xmin=0, xmax=1, c='red')

axs2[2].scatter(x_silverware, silverware_change, label='median: -10.99', c='green', alpha=0.6)
axs2[2].plot(x_silverware, silverware_change, c='black', alpha=0.5, linestyle='--')
axs2[2].axhline(y=0, xmin=0, xmax=1, c='red')

axs2[3].scatter(x_drawers, drawers_change, label='median: -8.45', c='darkmagenta', alpha=0.6)
axs2[3].plot(x_drawers, drawers_change, c='black', alpha=0.5, linestyle='--')
axs2[3].axhline(y=0, xmin=0, xmax=1, c='red')

plt.suptitle('Comparison of simulation variants (automatica):\n percentual change from baseline to prediction', 
             fontsize=12, y=0.96)
axs2[3].set_xticks(x_original)
axs2[3].set_xticklabels(IDs, rotation=90, fontsize=8)

for ax in axs2.flat:
    ax.yaxis.set_major_formatter(PercentFormatter(100))
    ax.set_yticks(ticks=[-100,-75,-50,-25,0,25])

axs2[0].set_ylabel('original', fontsize=10)
axs2[1].set_ylabel('no duplicates', fontsize=10)
axs2[2].set_ylabel('silverware', fontsize=10)
axs2[3].set_ylabel('drawers', fontsize=10)

axs2[3].set_xlabel('sequences', fontsize=10)

axs2[0].legend(loc='lower left')
axs2[1].legend()
axs2[2].legend()
axs2[3].legend()

plt.ylim(-150,55)
plt.show()